# Tokenization

In [61]:
import os
import urllib.request
file_path = "../data/the-verdict.txt"
if not os.path.exists(file_path):
  url = "https://raw.githubusercontent.com/rasbt/LLMs-from-scratch/main/ch02/01_main-chapter-code/the-verdict.txt"
  urllib.request.urlretrieve(url, file_path)

with open("../data/the-verdict.txt", "r", encoding="utf-8") as f:
  raw_text = f.read()  
print("Total number of character:", len(raw_text))
print(raw_text[:99])

Total number of character: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


# 2.1 Building a Simple Tokenizer - with regex | With Example

In [66]:
import re

#text = "Hello, world. This, is a test.?"

text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."
text = " <|endoftext|> ".join((text1, text2))

result = re.split(r'([,.:;?_!"()\']|--|\s)', text)
result = [item for item in result if item.strip()]
print("Text :", result)

# 2.2 Converting tokens into token IDs

print("\n 2.2 Converting tokens into token IDs \n")
all_words = sorted(set(result))
vocab_size = len(all_words)
print("Vocab Size : ",vocab_size)
vocab = {token:integer for integer,token in enumerate(all_words)}

# 2.3 Putting it now all together into a tokenizer class

class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}
    
    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
                                
        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
        
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text


tokenizer = SimpleTokenizerV1(vocab)
ids = tokenizer.encode(text)
print("Tokens Ids: ",ids)
print("Decoded Tokens: ",tokenizer.decode(ids))


# 2.4 Adding special context tokens - It's useful to add some "special" tokens for unknown words and to denote the end of a text

print("\n 2.4 Adding special context tokens \n")
all_tokens = sorted(list(set(result)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])

vocab = {token:integer for integer,token in enumerate(all_tokens)}
print("Vocab Size -",len(vocab.items()))


class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = { i:s for s,i in vocab.items()}
    
    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed = [
            item if item in self.str_to_int 
            else "<|unk|>" for item in preprocessed
        ]

        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
        
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.:;?!"()\'])', r'\1', text)
        return text
    


tokenizer = SimpleTokenizerV2(vocab)
print("Tokens Ids : ", tokenizer.encode(text))
print("Decoded Tokens :",tokenizer.decode(tokenizer.encode(text)))


# 2.5 BytePair encoding 
# GPT-2 used BytePair encoding (BPE) as its tokenizer
# it allows the model to break down words that aren't in its predefined vocabulary into smaller subword units or even individual characters, enabling it to handle out-of-vocabulary words
# For instance, if GPT-2's vocabulary doesn't have the word "unfamiliarword," it might tokenize it as ["unfam", "iliar", "word"] or some other subword breakdown, depending on its trained BPE merges
# In this chapter, we are using the BPE tokenizer from OpenAI's open-source tiktoken library,



print("\n \n2.5 BytePair encoding \n")
import importlib
import tiktoken

print("tiktoken version:", importlib.metadata.version("tiktoken"))
tokenizer = tiktoken.get_encoding("gpt2")


text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
    "of someunknownPlace."
)

ids = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
print("Tokens Ids :  ", ids)
print("Decoded Tokens :",tokenizer.decode(ids))


# 2.6 Data sampling with a sliding window

print("\n \n2.6 Data sampling with a sliding window \n")
enc_sample = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
print("Tokens Ids :  ", enc_sample)
context_size = 4

x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]

print(f"x: {x}")
print(f"y:      {y}")
print("\n")
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]
    print(context, "---->", desired, " || ", tokenizer.decode(context), "---->", tokenizer.decode([desired]))
print("\n")

print("We implement a simple data loader that iterates over the input dataset and returns the inputs and targets shifted by one")
import torch
print("PyTorch version:", torch.__version__)



print(" \nCreate dataset and dataloader that extract chunks from the input text dataset")

from torch.utils.data import Dataset, DataLoader

class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        # Tokenize the entire text
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})
        assert len(token_ids) > max_length, "Number of tokenized inputs must at least be equal to max_length+1"

        # Use a sliding window to chunk the book into overlapping sequences of max_length
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]
    
def create_dataloader_v1(txt, batch_size=4, max_length=256, stride=128, shuffle=True, drop_last=True,num_workers=0):

    # Initialize the tokenizer
    tokenizer = tiktoken.get_encoding("gpt2")

    # Create dataset
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    # Create dataloader
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )

    return dataloader

dataloader = create_dataloader_v1(
    text, batch_size=1, max_length=4, stride=1, shuffle=False
)


# More overlap could lead to increased overfitting

dataloader = create_dataloader_v1(text, batch_size=3, max_length=4, stride=4, shuffle=False)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\n", targets)


print("\n \n 2.7 Creating token embeddings")
output_dim = 3
token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)
print(token_embedding_layer.weight)




Text : ['Hello', ',', 'do', 'you', 'like', 'tea', '?', '<|endoftext|>', 'In', 'the', 'sunlit', 'terraces', 'of', 'the', 'palace', '.']

 2.2 Converting tokens into token IDs 

Vocab Size :  15
Tokens Ids:  [4, 0, 6, 14, 7, 11, 3, 2, 5, 13, 10, 12, 8, 13, 9, 1]
Decoded Tokens:  Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.

 2.4 Adding special context tokens 

Vocab Size - 16
Tokens Ids :  [4, 0, 6, 14, 7, 11, 3, 15, 5, 13, 10, 12, 8, 13, 9, 1]
Decoded Tokens : Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.

 
2.5 BytePair encoding 

tiktoken version: 0.11.0
Tokens Ids :   [15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271, 13]
Decoded Tokens : Hello, do you like tea? <|endoftext|> In the sunlit terracesof someunknownPlace.

 
2.6 Data sampling with a sliding window 

Tokens Ids :   [15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 6